In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.linear_model import LassoCV, RidgeCV, MultiTaskLassoCV, MultiTaskElasticNetCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import make_scorer, r2_score

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

import gc
import sys

df = pd.read_csv("data.csv")
df


,KS4_GPA5_Qtg.PV,KS4_GPA5_TC_T1.PV,KS4_GPA5_Nst.PV,KS4_GPA5_Nvd.PV,KS4_GPA5_Nnd.PV,KS4_GPA5_T4.PV,KS4_GPA5_Pk.PV
0,50.074285,50.656771,500.475537,1207.542773,912.161426,119.562970,51.248955
1,50.074926,50.656771,500.379883,1209.190723,914.203027,119.683643,51.255357
2,50.074836,50.666920,499.359570,1210.159082,914.924512,119.665576,51.263570
3,50.074957,50.666920,500.535840,1210.641602,915.881055,119.674750,51.267725
4,50.075145,50.666920,500.198145,1211.398438,916.435645,119.692188,51.264826
...,...,...,...,...,...,...,...
29707,50.070496,52.706500,459.827393,1238.176660,914.571973,128.579962,51.102349
29708,50.070217,52.716716,459.400806,1237.532422,913.476074,128.521655,51.102543
29709,50.070292,52.696284,459.942578,1237.783984,913.913086,128.503632,51.102639
29710,50.070389,52.675853,460.185791,1237.441895,913.895508,128.481744,51.103799


In [34]:
inputs = ['KS4_GPA5_Qtg.PV', 'KS4_GPA5_TC_T1.PV', 'KS4_GPA5_Nst.PV']
outputs = ['KS4_GPA5_Nvd.PV', 'KS4_GPA5_Nnd.PV', 'KS4_GPA5_T4.PV', 'KS4_GPA5_Pk.PV']

In [35]:
for i in inputs + outputs:
    meanVal = df[i].mean()
    df[i].fillna(meanVal, inplace=True)

df.describe()

,KS4_GPA5_Qtg.PV,KS4_GPA5_TC_T1.PV,KS4_GPA5_Nst.PV,KS4_GPA5_Nvd.PV,KS4_GPA5_Nnd.PV,KS4_GPA5_T4.PV,KS4_GPA5_Pk.PV
count,29712.000000,29712.000000,29712.000000,29712.000000,29712.000000,29712.000000,29712.000000
mean,50.075761,51.582940,496.375057,1228.936047,925.288586,123.147985,51.579620
std,0.004490,0.718957,16.656003,9.293723,12.216246,2.293297,58.430508
min,50.067757,50.118885,458.647290,1207.311621,910.000195,118.375299,51.074956
25%,50.072440,50.940938,484.903076,1222.372168,915.377100,121.428400,51.183080
50%,50.074476,51.725804,490.411133,1228.801709,921.373828,123.233508,51.244438
75%,50.079485,52.134428,509.792322,1236.431030,934.868530,125.019626,51.271976
max,50.089918,53.053829,526.688867,1264.442773,977.826953,128.815308,10123.000000


In [36]:
df['KS4_GPA5_Nnd.PV'] = df['KS4_GPA5_Nnd.PV'].fillna(df['KS4_GPA5_Nnd.PV'].median())
df

,KS4_GPA5_Qtg.PV,KS4_GPA5_TC_T1.PV,KS4_GPA5_Nst.PV,KS4_GPA5_Nvd.PV,KS4_GPA5_Nnd.PV,KS4_GPA5_T4.PV,KS4_GPA5_Pk.PV
0,50.074285,50.656771,500.475537,1207.542773,912.161426,119.562970,51.248955
1,50.074926,50.656771,500.379883,1209.190723,914.203027,119.683643,51.255357
2,50.074836,50.666920,499.359570,1210.159082,914.924512,119.665576,51.263570
3,50.074957,50.666920,500.535840,1210.641602,915.881055,119.674750,51.267725
4,50.075145,50.666920,500.198145,1211.398438,916.435645,119.692188,51.264826
...,...,...,...,...,...,...,...
29707,50.070496,52.706500,459.827393,1238.176660,914.571973,128.579962,51.102349
29708,50.070217,52.716716,459.400806,1237.532422,913.476074,128.521655,51.102543
29709,50.070292,52.696284,459.942578,1237.783984,913.913086,128.503632,51.102639
29710,50.070389,52.675853,460.185791,1237.441895,913.895508,128.481744,51.103799


In [37]:
#Lasso
x, y = df[inputs].to_numpy(), df[outputs].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=1/3, random_state=0, shuffle=True)

lassoRegression = MultiTaskLassoCV(cv=5, random_state=0, alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10])
lassoRegression.fit(x_train, y_train)

print(f'Тестовые данные: {lassoRegression.score(x_test, y_test)}')
print(f'Тренировочные данные: {lassoRegression.score(x_train, y_train)}')

Тестовые данные: 0.6419523458695507
Тренировочные данные: 0.8755686892352593


In [38]:
#Ridge
ridge_regressor = RidgeCV(cv=5, alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10])
ridge_regressor.fit(x_train, y_train)

print(f'Тестовые данные: { ridge_regressor.score(x_test, y_test)}')
print(f'Тренировочные данные: {ridge_regressor.score(x_train, y_train)}')

Тестовые данные: 0.641976331469716
Тренировочные данные: 0.8755807798925561


In [39]:
#ElasticNet
elastic_net = MultiTaskElasticNetCV(alphas=[0.0001, 0.001,0.01, 0.1, 1, 10], l1_ratio=[0.01, 0.1, 0.5, 1, 5, 10, 15, 20], cv=5)
elastic_net.fit(x_train, y_train)

print(f'Тестовые данные: {elastic_net.score(x_test, y_test)}')
print(f'Тренировочные данные: {elastic_net.score(x_train, y_train)}')

C:\Users\ibnee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:606: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 191.47798540458376
  model = cd_fast.enet_coordinate_descent_multi_task(
C:\Users\ibnee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1403: RuntimeWarning: overflow encountered in square
  this_mse = (residues**2).mean(axis=0)
C:\Users\ibnee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:606: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 191.46922145126587
  model = cd_fast.enet_coordinate_descent_multi_task(
C:\Users\ibnee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1403: RuntimeWarning: o

Тестовые данные: 0.6419523458695507
Тренировочные данные: 0.8755686892352593


C:\Users\ibnee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:606: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 46737.29659670929, tolerance: 190.60137058914094
  model = cd_fast.enet_coordinate_descent_multi_task(


In [40]:
#DecisionTreeRegressor
grid_search_cv = GridSearchCV(DecisionTreeRegressor(), param_grid={'max_features': [1, 2, 3]})
grid_search_cv.fit(x_train, y_train)
decision_tree = grid_search_cv.best_estimator_

print(f'Тестовые данные: {decision_tree.score(x_test, y_test)}')
print(f'Тренировочные данные: {decision_tree.score(x_train, y_train)}')

Тестовые данные: 0.7364742023885992
Тренировочные данные: 0.9999999666944049


In [41]:
# SVR
grid_search = GridSearchCV(MultiOutputRegressor(SVR()), cv=5, param_grid={
     'estimator__gamma': [0.001, 0.01],
 'estimator__C': [0.001, 0.01],
})
grid_search.fit(x_train, y_train)
svr_regressor = grid_search.best_estimator_

print(f'Тестовые данные: {svr_regressor.score(x_test, y_test)}')
print(f'Тренировочные данные: {svr_regressor.score(x_train, y_train)}')

Тестовые данные: 0.3064149476225815
Тренировочные данные: 0.4550449725663216


In [42]:
# Random Forest Regressor
grid_search_forest = GridSearchCV(RandomForestRegressor(), param_grid={
    'max_features': [1, 2, 3]
})
grid_search_forest.fit(x_train, y_train)
forest_regressor = grid_search_forest.best_estimator_

print(f'Тестовые данные: {forest_regressor.score(x_test, y_test)}')
print(f'Тренировочные данные: {forest_regressor.score(x_train, y_train)}')

Тестовые данные: 0.7426543357488069
Тренировочные данные: 0.9988353484949275
